In [2]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.functions import col


import os

In [3]:
os.getcwd()

'/Users/josep/Documents/FlatsHousesStudyBCN'

In [4]:
spark = SparkSession.builder.appName("Practise").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/01 16:32:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark

## Basics

In [6]:
csv_options = {
    "header": True,               # Si tu archivo CSV tiene encabezados de columna
    "multiLine": True,            # Permite que los valores del CSV se extiendan en varias líneas
    "escape": '"',                # Carácter de escape para delimitar los campos que pueden contener saltos de línea
    "quote": '"',                 # Carácter de comillas para delimitar los campos
    "ignoreLeadingWhiteSpace": True,  # Ignorar espacios en blanco al principio de los valores del CSV
    "ignoreTrailingWhiteSpace": True,  # Ignorar espacios en blanco al final de los valores del CSV
    # "schema":schema
    "inferSchema":True
}

# schema = StructType([
    # StructField("id", StringType(), True),
    # StructField("level8", StringType(), True),
    # StructField("value", IntegerType(), True),
# ])
# df = spark.read.option('header','true').csv("data1.csv",inferSchema=True)
df = spark.read.csv("data210_310.csv",  **csv_options) # , inferSchema=True, header= 'true', schema=schema)

In [7]:
# df.columns
# df.head(4)
df = df.select(["id", "level7", "value"])
df.show()

24/03/01 16:32:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, type, subtype, external_contract, invalid, new, advertiser, description, transactions, transaction_type, value, reduced, periodicity_id, ubication, zipcode, highlight, date, energy_letter, energy_value, environment_letter, environment_value, multimedia_type_2, multimedia_type_6, multimedia_type_12, multimedia_type_21, multimedia_type_5, rooms, bathrooms, surface, energyCertificate, orientation, hotWater, heating, conservationState, antiquity, floor, surfaceLand, orientation, conservationState, antiquity, furnished, hotWater, heating, floor, elevator, parking, Parquet, Cocina Office, Horno, Balcón, Gres Cerámica, Terraza, Patio, Lavadero, Cocina Equipada, Electrodomésticos, Lavadora, Microondas, Nevera, TV, Aire acondicionado, Armarios, Suite - con baño, Trastero, Puerta Blindada, Calefacción, Internet, Jardín Privado, Serv. portería, Piscina, Z. Comunitaria, Zona Infantil, Piscina comunitar

+---------+--------------------+-------+
|       id|              level7|  value|
+---------+--------------------+-------+
|181354183|        Ciutat Vella| 310000|
|181368065|          Sant Martí| 373000|
|181628787|            Eixample| 596000|
|181607356|           Les Corts| 530000|
|178670316|           Les Corts| 490000|
|179103362|    Horta - Guinardó| 490000|
|176317665|          Nou Barris| 250000|
|176315153|         Sant Andreu| 423000|
|181778859|    Horta - Guinardó| 800000|
|177868160|        Ciutat Vella| 315000|
|181529802|    Sants - Montjuïc| 199000|
|181568183|            Eixample| 500000|
|178177353|           Les Corts|1250000|
|181402728|          Nou Barris| 255000|
|178353774|           Les Corts|1250000|
|179759313|        Ciutat Vella|1101400|
|179688782|            Eixample| 860000|
|179025853|          Sant Martí| 239000|
|181531162|Sarrià - Sant Ger...| 500000|
|181820040|    Horta - Guinardó| 430000|
+---------+--------------------+-------+
only showing top

In [8]:
# df.show()
# df.printSchema()
# df.dtypes
df.describe().show()

24/03/01 16:32:29 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, type, subtype, external_contract, invalid, new, advertiser, description, transactions, transaction_type, value, reduced, periodicity_id, ubication, zipcode, highlight, date, energy_letter, energy_value, environment_letter, environment_value, multimedia_type_2, multimedia_type_6, multimedia_type_12, multimedia_type_21, multimedia_type_5, rooms, bathrooms, surface, energyCertificate, orientation, hotWater, heating, conservationState, antiquity, floor, surfaceLand, orientation, conservationState, antiquity, furnished, hotWater, heating, floor, elevator, parking, Parquet, Cocina Office, Horno, Balcón, Gres Cerámica, Terraza, Patio, Lavadero, Cocina Equipada, Electrodomésticos, Lavadora, Microondas, Nevera, TV, Aire acondicionado, Armarios, Suite - con baño, Trastero, Puerta Blindada, Calefacción, Internet, Jardín Privado, Serv. portería, Piscina, Z. Comunitaria, Zona Infantil, Piscina comunitar

+-------+--------------------+--------------------+-----------------+
|summary|                  id|              level7|            value|
+-------+--------------------+--------------------+-----------------+
|  count|                2955|                2955|             2955|
|   mean|1.7856323906328258E8|                NULL|556157.8524534687|
| stddev|   5600043.911564616|                NULL| 527055.974744297|
|    min|           144318805|        Ciutat Vella|            39900|
|    max|           181972113|Sarrià - Sant Ger...|          6000000|
+-------+--------------------+--------------------+-----------------+



In [9]:
# df["id"]
# df.select("id").show()

In [10]:
df = df.withColumn("id", df["id"].cast("int"))
df = df.withColumn("value_inflation_1year", df["value"]+(df["value"]*0.03))
# df.drop("value").show()
# df.show()

df.filter(df["value"]<200000).show()

+---------+----------------+------+---------------------+
|       id|          level7| value|value_inflation_1year|
+---------+----------------+------+---------------------+
|181529802|Sants - Montjuïc|199000|             204970.0|
|181798114|    Ciutat Vella|169000|             174070.0|
|181753991|Sants - Montjuïc|139000|             143170.0|
|181718985|    Ciutat Vella|199000|             204970.0|
|181420837|     Sant Andreu|199900|             205897.0|
|179249141|Horta - Guinardó| 70000|              72100.0|
|177613697|Sants - Montjuïc|199900|             205897.0|
|179654634|    Ciutat Vella|155000|             159650.0|
|178454276|    Ciutat Vella|175000|             180250.0|
|181562182|Sants - Montjuïc|199000|             204970.0|
|181389319|      Sant Martí| 95000|              97850.0|
|163449744|    Ciutat Vella|190000|             195700.0|
|181266594|Sants - Montjuïc|185000|             190550.0|
|176650359|Horta - Guinardó|142000|             146260.0|
|176944365|   

24/03/01 16:32:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, type, subtype, external_contract, invalid, new, advertiser, description, transactions, transaction_type, value, reduced, periodicity_id, ubication, zipcode, highlight, date, energy_letter, energy_value, environment_letter, environment_value, multimedia_type_2, multimedia_type_6, multimedia_type_12, multimedia_type_21, multimedia_type_5, rooms, bathrooms, surface, energyCertificate, orientation, hotWater, heating, conservationState, antiquity, floor, surfaceLand, orientation, conservationState, antiquity, furnished, hotWater, heating, floor, elevator, parking, Parquet, Cocina Office, Horno, Balcón, Gres Cerámica, Terraza, Patio, Lavadero, Cocina Equipada, Electrodomésticos, Lavadora, Microondas, Nevera, TV, Aire acondicionado, Armarios, Suite - con baño, Trastero, Puerta Blindada, Calefacción, Internet, Jardín Privado, Serv. portería, Piscina, Z. Comunitaria, Zona Infantil, Piscina comunitar

In [11]:
df.withColumnRenamed("id","id_flat").show()

+---------+--------------------+-------+---------------------+
|  id_flat|              level7|  value|value_inflation_1year|
+---------+--------------------+-------+---------------------+
|181354183|        Ciutat Vella| 310000|             319300.0|
|181368065|          Sant Martí| 373000|             384190.0|
|181628787|            Eixample| 596000|             613880.0|
|181607356|           Les Corts| 530000|             545900.0|
|178670316|           Les Corts| 490000|             504700.0|
|179103362|    Horta - Guinardó| 490000|             504700.0|
|176317665|          Nou Barris| 250000|             257500.0|
|176315153|         Sant Andreu| 423000|             435690.0|
|181778859|    Horta - Guinardó| 800000|             824000.0|
|177868160|        Ciutat Vella| 315000|             324450.0|
|181529802|    Sants - Montjuïc| 199000|             204970.0|
|181568183|            Eixample| 500000|             515000.0|
|178177353|           Les Corts|1250000|            128

24/03/01 16:32:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, type, subtype, external_contract, invalid, new, advertiser, description, transactions, transaction_type, value, reduced, periodicity_id, ubication, zipcode, highlight, date, energy_letter, energy_value, environment_letter, environment_value, multimedia_type_2, multimedia_type_6, multimedia_type_12, multimedia_type_21, multimedia_type_5, rooms, bathrooms, surface, energyCertificate, orientation, hotWater, heating, conservationState, antiquity, floor, surfaceLand, orientation, conservationState, antiquity, furnished, hotWater, heating, floor, elevator, parking, Parquet, Cocina Office, Horno, Balcón, Gres Cerámica, Terraza, Patio, Lavadero, Cocina Equipada, Electrodomésticos, Lavadora, Microondas, Nevera, TV, Aire acondicionado, Armarios, Suite - con baño, Trastero, Puerta Blindada, Calefacción, Internet, Jardín Privado, Serv. portería, Piscina, Z. Comunitaria, Zona Infantil, Piscina comunitar

## Nulls

In [12]:
df.na.drop(how="any", subset=["value"]).show() # how="all" to only drop if all are null, thresh=...


+---------+--------------------+-------+---------------------+
|       id|              level7|  value|value_inflation_1year|
+---------+--------------------+-------+---------------------+
|181354183|        Ciutat Vella| 310000|             319300.0|
|181368065|          Sant Martí| 373000|             384190.0|
|181628787|            Eixample| 596000|             613880.0|
|181607356|           Les Corts| 530000|             545900.0|
|178670316|           Les Corts| 490000|             504700.0|
|179103362|    Horta - Guinardó| 490000|             504700.0|
|176317665|          Nou Barris| 250000|             257500.0|
|176315153|         Sant Andreu| 423000|             435690.0|
|181778859|    Horta - Guinardó| 800000|             824000.0|
|177868160|        Ciutat Vella| 315000|             324450.0|
|181529802|    Sants - Montjuïc| 199000|             204970.0|
|181568183|            Eixample| 500000|             515000.0|
|178177353|           Les Corts|1250000|            128

24/03/01 16:32:32 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, type, subtype, external_contract, invalid, new, advertiser, description, transactions, transaction_type, value, reduced, periodicity_id, ubication, zipcode, highlight, date, energy_letter, energy_value, environment_letter, environment_value, multimedia_type_2, multimedia_type_6, multimedia_type_12, multimedia_type_21, multimedia_type_5, rooms, bathrooms, surface, energyCertificate, orientation, hotWater, heating, conservationState, antiquity, floor, surfaceLand, orientation, conservationState, antiquity, furnished, hotWater, heating, floor, elevator, parking, Parquet, Cocina Office, Horno, Balcón, Gres Cerámica, Terraza, Patio, Lavadero, Cocina Equipada, Electrodomésticos, Lavadora, Microondas, Nevera, TV, Aire acondicionado, Armarios, Suite - con baño, Trastero, Puerta Blindada, Calefacción, Internet, Jardín Privado, Serv. portería, Piscina, Z. Comunitaria, Zona Infantil, Piscina comunitar

In [13]:
# Filling nulls
# df.na.fill("NAN")
# df.na.fill("nan", ["value","value_inflation_1year"])

## Group By

In [14]:
df_grouped = df.groupby("level7").agg({"id":"count","value":"mean","value_inflation_1year":"mean"})

In [15]:
df_grouped.withColumn("diff",df_grouped["avg(value_inflation_1year)"]-df_grouped["avg(value)"]).show()

24/03/01 16:32:32 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, type, subtype, external_contract, invalid, new, advertiser, description, transactions, transaction_type, value, reduced, periodicity_id, ubication, zipcode, highlight, date, energy_letter, energy_value, environment_letter, environment_value, multimedia_type_2, multimedia_type_6, multimedia_type_12, multimedia_type_21, multimedia_type_5, rooms, bathrooms, surface, energyCertificate, orientation, hotWater, heating, conservationState, antiquity, floor, surfaceLand, orientation, conservationState, antiquity, furnished, hotWater, heating, floor, elevator, parking, Parquet, Cocina Office, Horno, Balcón, Gres Cerámica, Terraza, Patio, Lavadero, Cocina Equipada, Electrodomésticos, Lavadora, Microondas, Nevera, TV, Aire acondicionado, Armarios, Suite - con baño, Trastero, Puerta Blindada, Calefacción, Internet, Jardín Privado, Serv. portería, Piscina, Z. Comunitaria, Zona Infantil, Piscina comunitar

+--------------------+---------+--------------------------+------------------+------------------+
|              level7|count(id)|avg(value_inflation_1year)|        avg(value)|              diff|
+--------------------+---------+--------------------------+------------------+------------------+
|              Gràcia|      193|         546475.8393782383|  530559.067357513|  15916.7720207253|
|          Sant Martí|      355|         525557.6943943661|510250.18873239437| 15307.50566197175|
|           Les Corts|      140|         825063.1071428572| 801032.1428571428| 24030.96428571432|
|    Horta - Guinardó|      205|         368190.9597560976| 357466.9512195122|10724.008536585374|
|          Nou Barris|      224|        246194.78437500002|       239024.0625|7170.7218750000175|
|Sarrià - Sant Ger...|      264|        1084422.1568181817|1052837.0454545454| 31585.11136363633|
|            Eixample|      598|         812800.5671404682| 789126.7642140468|  23673.8029264214|
|    Sants - Montjuï

In [16]:
type(df_grouped)

pyspark.sql.dataframe.DataFrame

## Joins and concat

In [17]:
# joined_df = df1.join(df2, df1["common_column"] == df2["common_column"], how="inner")
# joined_df = df1.join(df2, (df1["column1"] == df2["columnA"]) & (df1["column2"] == df2["columnB"]), how="inner")


In [18]:
df1 = spark.read.csv("data210_310.csv",  **csv_options).select(["id", "level7", "value"]) # , inferSchema=True, header= 'true', schema=schema)
df2 = spark.read.csv("data310_410.csv",  **csv_options).select(["id", "level7", "value"]) # , inferSchema=True, header= 'true', schema=schema)

In [19]:
concatenated_df = df1.union(df2)
#df = df.select(["id", "level7", "value"])


In [20]:
concatenated_df.dropDuplicates().show()

24/03/01 16:32:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, type, subtype, external_contract, invalid, new, advertiser, description, transactions, transaction_type, value, reduced, periodicity_id, ubication, zipcode, highlight, date, energy_letter, energy_value, environment_letter, environment_value, multimedia_type_2, multimedia_type_21, multimedia_type_6, multimedia_type_12, multimedia_type_5, rooms, bathrooms, surface, energyCertificate, orientation, hotWater, heating, conservationState, antiquity, floor, surfaceLand, floor, elevator, orientation, conservationState, antiquity, furnished, parking, heating, hotWater, Aire acondicionado, Terraza, Balcón, Calefacción, Armarios, Electrodomésticos, Puerta Blindada, Cocina Equipada, Parquet, Lavadero, Ascensor interior, Cocina Office, Suite - con baño, Patio, Gres Cerámica, Lavadora, Microondas, Nevera, TV, Horno, Trastero, Baño de huéspedes, Jardín Privado, Piscina, Jacuzzi, Gimnasio, Sauna, Z. Comunit

+---------+--------------------+-------+
|       id|              level7|  value|
+---------+--------------------+-------+
|181368065|          Sant Martí| 373000|
|181566056|              Gràcia| 670000|
|181284469|              Gràcia| 385000|
|181096441|        Ciutat Vella| 335000|
|181232791|              Gràcia| 234000|
|179374130|            Eixample| 490000|
|181424830|          Nou Barris| 140000|
|181346737|    Sants - Montjuïc| 200000|
|179284990|            Eixample| 380000|
|165191869|            Eixample|1150000|
|181912046|            Eixample| 289000|
|177451252|          Sant Martí| 399000|
|178731969|Sarrià - Sant Ger...|2315000|
|181237107|        Ciutat Vella| 242000|
|181564678|          Sant Martí| 238000|
|179749928|    Sants - Montjuïc| 300000|
|181438056|          Sant Martí| 990000|
|181568183|            Eixample| 500000|
|181566171|           Les Corts| 530000|
|178985562|Sarrià - Sant Ger...| 850000|
+---------+--------------------+-------+
only showing top